# FP-Tree
**F**requent **P**attern **Tree**s stellen eine weitere Möglichkeit dar, um nach häufigen Itemsets zu suchen. Sie erzeugen eine spezielle Datenstruktur, die das aufwendige Konstruieren geeigneter Kandidaten vermeidet. Durch einen darauf angepassten Mining-Algorithmus lassen sich häufig auftauchende Kombinationen aus dem Baum ohne erneute Scans der Datenbank ableiten.

In [ ]:
from tui_dsmt.fpm import FPTree, characters2

characters2

## Inhaltsverzeichnis
- [Aufbau des FP-Trees](#Aufbau-des-FP-Trees)
- [Mining des Baumes](#Mining-des-Baumes)
- [Zusammenfassung](#Zusammenfassung)

## Aufbau des FP-Trees
Der FP-Tree ist eine spezielle Datenstruktur, die den Inhalt einer Transaktionsdatenbank komprimiert, aber vollständig abspeichert. Irrelevante Informationen bezüglich nicht-häufiger Items und mehrfach auftretender Kombinationen werden zeitig verworfen, sodass Komprimierungsfaktoren bis $100$ möglich sind. Der Vorteil gegenüber den bisher bekannten Verfahren ist die Vermeidung mehrfacher Scans der Datenbank und der Generierung viele potentieller Kandidaten.

Wie bereits zuvor ist ein minimaler Supportwert nötig, der zu Beginn des Algorithmus bekannt sein muss.

In [ ]:
min_supp = 0.5 * len(characters2)
min_supp

Die Konstruktion des FP-Trees besteht aus drei Schritten:
1. Zuerst werden häufige Itemsets der Länge $1$ in den Transaktionen gesucht.
2. Die Items werden absteigend nach ihrer Häufigkeit sortiert und mit Hilfe des minimalen Supports gefiltert.
3. Die Datenbank wird ein zweites Mal durchsucht und der FP-Tree aufgebaut.

### Suche nach 1-Itemsets
Zuerst werden alle $1$-Itemsets bestimmt und ihre Häufigkeit gezählt. Dazu muss einmal über alle Transaktionen iteriert werden.

In [ ]:
counts = {}
for _, itemset in characters2:
    for element in itemset:
        counts[element] = counts.get(element, 0) + 1

counts

### Filtern und Sortieren
Die gefundenen $1$-Itemsets werden absteigend nach ihrer Häufigkeit sortiert und anhand des vorgegebenen, minimalen Supports gefiltert.

In [ ]:
ordered_frequent_1_itemsets = sorted((itemset for itemset, count in counts.items() if count >= min_supp),
                                     key=lambda x: (counts[x], x), reverse=True)
ordered_frequent_1_itemsets

### Aufbau des Baumes und der Header-Tabelle
Der Aufbau des FP-Trees besteht nun aus aus dem Füllen des Baumes und der gleichzeitigen Pflege einer Header-Tabelle.
Der FP-Tree wird mit dem leeren Element als Wurzel initialisiert, während die Header-Tabelle leer ist.

Anschließend wird die Datenbank ein zweites Mal gescannt. Dabei werden aus jeder Transaktion genau die Elemente ausgewählt, die sich ebenfalls in `ordered_frequent_1_itemsets` befinden, und anschließend nach ihrer Häufigkeit absteigend sortiert. Die verbleibenden Teilmengen jeder Transaktion werden ausgehend vom leeren Wurzelelement in den Baum eingetragen. Jeder Knoten erhält zudem einen Zähler, um festzustellen, in wie vielen Transaktionen der jeweilige Pfad vorkommt.

In [ ]:
fp_tree = FPTree(characters2, min_supp, custom_item_order=('f', 'c', 'a', 'b', 'm', 'p'))
fp_tree

Parallel zum Aufbau des Baumes wird die rechtsseitig abgebildete Header-Tabelle gepflegt. In ihr befinden sich zeilenweise die 1-Itemsets, ihre (aktuelle) summierte Häufigkeit im Baum und eine Liste von Referenzen auf Knoten innerhalb des Baumes. Die Verbindung der jeweiligen Items ist innerhalb des Graphen zusätzlich durch gepunktete Kanten markiert.

Die gespeicherten Verbindungen der Knoten werden beim Mining des Baumes relevant.

### Eigenschaften des FP-Trees
Den FP-Tree charakterisieren zwei spezifische Eigenschaften:
1. Die **Knoten-Link-Eigenschaft** besagt, dass für jedes häufige Item alle Muster, die dieses Item enthalten, über die von dem zugehörigen Knoten ausgehenden Kanten ermittelt werden können. (Durch die Verlinkung der bspw. $b$-Knoten untereinander können effizient alle Muster gefunden werden, die $b$ enthalten.)
2. Die **Präfix-Pfad-Eigenschaft** besagt, dass zur Bestimmung der häufigen Muster für einen Knoten im Pfad nur die Präfix-Subpfade gesammelt werden müssen. Die Häufigkeit entspricht dann der Häufigkeit des Knotens. (So kommt $p$ zwei mal mit $f$, $c$, $a$ und $m$ und ein mal mit $c$ und $b$ vor.)

## Mining des Baumes
Um die häufigen Itemsets zu finden, müssen die im FP-Tree gesammelten Informationen verarbeitet werden. Dafür wird der rekursive Algorithmus FP-Growth verwendet:
- Für jedes Item wird eine konditionale Musterbasis und anschließend ein konditionaler FP-Tree erstellt.
- Der Prozess wird für jeden erzeugten konditionalen FP-Tree wiederholt.
- Sobald der resultierende FP-Tree leer ist oder nur einen Pfad enthält, ist das Mining abgeschlossen.

Im Folgenden soll der Prozess stellvertretend für das Item `m` gezeigt werden.

### Konditionale Musterbasis
Zuerst muss eine konditionale Musterbasis gebildet werden. Dazu wird die zuvor angelegte Header-Tabelle benutzt, um für jedes Item alle Präfixe zu sammeln und zu zählen.

In [ ]:
cpb = fp_tree.conditional_pattern_base
cpb

### Konditionaler FP-Tree
Aus der konditionalen Musterbasis kann nun für ein einzelnes Item ein konditionaler FP-Tree erzeugt werden. Dazu wird die gesammelte Musterbasis in einen neuen Baum überführt.

In [ ]:
m_cfp_tree = cpb.conditional_fp_tree('m')
m_cfp_tree

### Rekursives Mining
Das Mining wird dann rekursiv fortgesetzt.

In [ ]:
m_cpb = m_cfp_tree.conditional_pattern_base
m_cpb

In [ ]:
am_cfp_tree = m_cpb.conditional_fp_tree('a')
am_cfp_tree

In [ ]:
am_cpb = am_cfp_tree.conditional_pattern_base
am_cpb

In [ ]:
cam_fp_tree = am_cpb.conditional_fp_tree('c')
cam_fp_tree

In [ ]:
cam_cpb = cam_fp_tree.conditional_pattern_base
cam_cpb

In [ ]:
fcam_fp_tree = cam_cpb.conditional_fp_tree('f')
fcam_fp_tree

Ist ein leerer, konditionaler FP-Tree das Ergebnis, endet die Rekursion. Entlang dieses Pfades wurden damit die häufigen Itemsets `m`, `am`, `cam` und `fcam` gefunden. Probieren Sie ausgehend vom $m$-konditionalen FP-Tree auch andere Pfade aus, sodass Sie ebenfalls die Muster `fm`, `cm`, `fcm` und `fam` finden.

## Zusammenfassung
In diesem Notebook wurde ein Verfahren vorgestellt, das durch die geschickte Verwendung einer speziellen Datenstruktur keiner Kandidatengenerierung bedarf. Im Allgemeinen ist das Erstellen *und* Minen des FP-Tree schneller als die Verwendung des apriori-Algorithmus, auch wenn der Unterschied durch einen höheren, minimalen Support sinkt.